In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [4]:
df = fetch_hourly_rides(12)

2025-03-04 14:21:13,697 INFO: Initializing external client
2025-03-04 14:21:13,698 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 14:21:14,361 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215656
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.60s) 


In [5]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-04 04:00:00+00:00,209,0
1,2025-03-04 02:00:00+00:00,164,7
2,2025-03-04 04:00:00+00:00,53,0
3,2025-03-04 02:00:00+00:00,65,2
4,2025-03-04 04:00:00+00:00,113,2
...,...,...,...
4262,2025-03-04 05:00:00+00:00,35,1
4263,2025-03-04 06:00:00+00:00,145,1
4264,2025-03-04 15:00:00+00:00,81,0
4265,2025-03-04 16:00:00+00:00,251,0


In [6]:
df_pred = fetch_predictions(12)

2025-03-04 14:21:25,665 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 14:21:25,670 INFO: Initializing external client
2025-03-04 14:21:25,671 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-04 14:21:26,268 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215656
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (26.32s) 


In [7]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,37,0.0,2025-03-04 04:00:00+00:00
1,153,0.0,2025-03-04 04:00:00+00:00
2,94,0.0,2025-03-04 04:00:00+00:00
3,252,0.0,2025-03-04 04:00:00+00:00
4,181,6.0,2025-03-04 04:00:00+00:00
...,...,...,...
1250,97,1.0,2025-03-04 19:00:00+00:00
1251,198,0.0,2025-03-04 19:00:00+00:00
1252,20,0.0,2025-03-04 19:00:00+00:00
1253,214,0.0,2025-03-04 19:00:00+00:00


In [8]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [9]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2025-03-04 04:00:00+00:00,209,0,3.0
1,2025-03-04 04:00:00+00:00,53,0,0.0
2,2025-03-04 04:00:00+00:00,113,2,96.0
3,2025-03-04 04:00:00+00:00,122,0,0.0
4,2025-03-04 04:00:00+00:00,212,0,1.0
...,...,...,...,...
999,2025-03-04 14:00:00+00:00,66,0,1.0
1000,2025-03-04 05:00:00+00:00,212,0,1.0
1001,2025-03-04 14:00:00+00:00,194,0,0.0
1002,2025-03-04 05:00:00+00:00,35,1,1.0


In [10]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [11]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
201,2025-03-04 04:00:00+00:00,2,0,0.0,0.0
922,2025-03-04 05:00:00+00:00,2,0,0.0,0.0
552,2025-03-04 14:00:00+00:00,2,0,0.0,0.0
355,2025-03-04 16:00:00+00:00,2,0,0.0,0.0
34,2025-03-04 04:00:00+00:00,3,0,0.0,0.0
...,...,...,...,...,...
340,2025-03-04 16:00:00+00:00,262,111,82.0,-29.0
113,2025-03-04 04:00:00+00:00,263,8,55.0,47.0
426,2025-03-04 05:00:00+00:00,263,18,93.0,75.0
336,2025-03-04 14:00:00+00:00,263,146,96.0,-50.0


In [12]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
0,2025-03-04 04:00:00+00:00,209,0,3.0,3.0
1,2025-03-04 04:00:00+00:00,53,0,0.0,0.0
2,2025-03-04 04:00:00+00:00,113,2,96.0,94.0
3,2025-03-04 04:00:00+00:00,122,0,0.0,0.0
4,2025-03-04 04:00:00+00:00,212,0,1.0,1.0
...,...,...,...,...,...
999,2025-03-04 14:00:00+00:00,66,0,1.0,1.0
1000,2025-03-04 05:00:00+00:00,212,0,1.0,1.0
1001,2025-03-04 14:00:00+00:00,194,0,0.0,0.0
1002,2025-03-04 05:00:00+00:00,35,1,1.0,0.0


In [13]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [14]:
mae_by_hour["MAE"].mean()

13.822709163346612